# Analysis 4
## Objectives
- in each city in state AZ, find the most profitable streets
- in each city in state AZ, find the streets where can be a good place to find restaurants
- in each city in state AZ, find the most popular streets to drink and play
## Steps
- use business.json in data set
- get city and street information for each business
- find street with most business in each city
- find restaurants with catogory containing certain keywords
- in these restaurants, find those with more than 10 reviews and more than 4.0/5.0 average stars
- rank the streets by number of those restaurants
- find business with catogory containing certain keywords with enough reviews and stars
- generate CSV as result

In [2]:
%matplotlib inline

In [3]:
import glob
import os
import json
from pathlib import Path
from pandas import Series, DataFrame
import pandas as pd

In [4]:
p = Path(os.getcwd())
business_path = str(p.parent) + '/data/yelp_training_set/yelp_training_set_business.json'

In [5]:
import re

In [6]:
df = pd.DataFrame(columns=['city', 'street', 'type', 'count'])
play_keywords = ['Bars', 'Nightlife']
with open(business_path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        data = json.loads(line)
        review_count = data['review_count']
        stars = data['stars']
        if review_count < 10 or stars < 4.0:
            continue
        address = data['full_address'].split('\n')
        if not re.match(r'^[0-9]', address[0]):
            continue
        city = address[-1].split(',')[0]
        street = ' '.join(address[0].split(' ')[1:])
        categories = data['categories']
        if 'Restaurants' in categories:
            df = df.append(Series({'city': city, 'street': street, 'type': 'restaurant', 'count': 1}), ignore_index=True)
        if not set(play_keywords).isdisjoint(categories):
            df = df.append(Series({'city': city, 'street': street, 'type': 'nightlife', 'count': 1}), ignore_index=True)
        if 'Restaurants' not in categories and set(play_keywords).isdisjoint(categories):
            df = df.append(Series({'city': city, 'street': street, 'type': 'other', 'count': 1}), ignore_index=True)
df.head(5)

,city,street,type,count
0,Scottsdale,E 1st St,other,1.0
1,Scottsdale,N Scottsdale Rd,restaurant,1.0
2,Phoenix,E Cactus Rd,other,1.0
3,Tempe,S McAllister Ave,other,1.0
4,Phoenix,N Black Canyon Hwy,restaurant,1.0


In [8]:
total_df = df.groupby(['city', 'street']).sum().reset_index()
total_df.head(5)

,city,street,count
0,Ahwatukee,E Chandler Blvd,1.0
1,Ahwatukee,E Warner Rd,2.0
2,Anthem,N Daisy Mountain Rd,1.0
3,Anthem,W Anthem Way,3.0
4,Anthem,W Anthem Wy,1.0


In [10]:
profitable = total_df.groupby(['city']).max().reset_index()
profitable.head(5)

,city,street,count
0,Ahwatukee,E Warner Rd,2.0
1,Anthem,W Anthem Wy,3.0
2,Apache Junction,W Apache Trl,1.0
3,Avondale,W. Thomas Rd,7.0
4,Buckeye,W Main St,2.0


In [29]:
profitable.to_csv('ana_4/profitable_streets_in_AZ.csv', index = False, header = True)

In [32]:
restaurant_df = df[df['type'] == 'restaurant']
restaurant_df = restaurant_df.groupby(['city', 'street']).sum().reset_index().groupby(['city']).max().reset_index()
restaurant_df.head(5)

,city,street,count
0,Ahwatukee,E Warner Rd,1.0
1,Anthem,W Anthem Wy,2.0
2,Apache Junction,W Apache Trl,1.0
3,Avondale,W Rancho Santa Fe Blvd,7.0
4,Buckeye,W Main St,1.0


In [35]:
restaurant_df.to_csv('ana_4/restaurant_streets_in_AZ.csv', index = False, header = True)

In [36]:
nightlife_df = df[df['type'] == 'nightlife']
nightlife_df = nightlife_df.groupby(['city', 'street']).sum().reset_index().groupby(['city']).max().reset_index()
nightlife_df.to_csv('ana_4/nightlife_streets_in_AZ.csv', index = False, header = True)
nightlife_df.head(5)

,city,street,count
0,Ahwatukee,E Warner Rd,1.0
1,Buckeye,W Main St,1.0
2,Cave Creek,E. Cave Creek Rd.,1.0
3,Chandler,W Ray Rd,3.0
4,Fort McDowell,N Hiawatha Rd,1.0
